In [80]:
import numpy as np
import torch
import gym
from torch import nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
from torch.utils import tensorboard

In [81]:
# helper function to convert numpy arrays to tensors
def t(x):
    return torch.tensor(x, dtype=torch.float)

In [82]:
# Critic module
class CriticNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [83]:
env = gym.make("CartPole-v1")

In [84]:
critic_net = CriticNet()
critic_optim = torch.optim.SGD(critic_net.parameters(), lr=1e-3)

In [90]:
w = tensorboard.SummaryWriter()

for i in range(4000):
    done = False
    previous_state = env.reset()

    while not done:
        #probs = actor(t(state))
        #dist = torch.distributions.Categorical(probs=probs)
        #action = dist.sample()
        #prob_act = dist.log_prob(action)
        action = 0 #env.action_space.sample()
        
        #next_state, reward, done, info = env.step(action.detach().data.numpy())
        state, reward, done, info = env.step(action)
        advantage = reward + critic_net(t(state)).detach() - critic_net(t(previous_state))
        
        #w.add_scalar("loss/advantage", advantage, global_step=s)
        #w.add_scalar("actions/action_0_prob", dist.probs[0], global_step=s)
        #w.add_scalar("actions/action_1_prob", dist.probs[1], global_step=s)
        
        total_reward += reward
        previous_state = state
        
        critic_loss = advantage.pow(2).mean()
            
        if done:
            w.add_scalar("loss/critic_loss", critic_loss, global_step=i)
                
        critic_optim.zero_grad()
        critic_loss.backward()
        critic_optim.step()